In [1]:
#import all the jank
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests
import lxml

In [2]:
#get splinter to the webpage
#first set up splinter to use chrome by installing webdriver
#this turns out to be unneccesary
#exe_p = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **exe_p, headless=False)

# Obtain Information for Iterative Scraping 

In [3]:
#go to Whiskey Advocate

booze = 'https://www.whiskyadvocate.com/'

#browser.visit(booze)

In [4]:
#scrape the style dropdown menu into a list

response = requests.get(booze)

soup = bs(response.text)

styles = soup.find_all('option')


print(len(styles))

cleanSty = []
id_vals = []

for style in styles:
    #print(style['value'])
    cleanSty.append(style.text)
    id_vals.append(style['value'])

1301


In [5]:
#use a dataframe to number these for easy identification

stDict = {"dropdown_options": cleanSty, "option_value": id_vals}

st_df = pd.DataFrame(stDict)

pd.set_option('display.max_rows', 400)

#st_df.tail(400)

In [6]:
#cut the list down to just the relevant values
#I most likely need the id values because they are what actually plug in the URL for the search url

w_styles = st_df.iloc[1142:1203]['dropdown_options'].to_list()

w_ids = st_df.iloc[1142:1203]['option_value'].to_list()

w_ids

['68',
 '15',
 '31',
 '52',
 '27',
 '53',
 '42',
 '57',
 '55',
 '67',
 '17',
 '69',
 '3',
 '4',
 '7',
 '10',
 '63',
 '40',
 '49',
 '48',
 '47',
 '44',
 '41',
 '28',
 '60',
 '19',
 '8',
 '65',
 '37',
 '46',
 '71',
 '72',
 '70',
 '11',
 '50',
 '73',
 '45',
 '62',
 '26',
 '9',
 '56',
 '5',
 '58',
 '20',
 '64',
 '18',
 '21',
 '23',
 '61',
 '59',
 '1',
 '16',
 '29',
 '32',
 '39',
 '30',
 '2',
 '13',
 '33',
 '34',
 '66']

# Identify Location of Relevant Information in the Search Results Page

In [7]:
#I will iterate through this list when I'm ready to scrape my full dataset. 
#search_url works with browser command

search_url ='https://www.whiskyadvocate.com/ratings-reviews/?search=&submit=&brand_id=0&rating=0&price=0&category=0&styles_id=68&issue_id=0'

#browser.visit(search_url)

In [8]:
#Now I need to figure out how to scrape the crap off the page at the test url. the first style has 8 entries 
#I should get 8 entries in the lists. 

chow = requests.get(search_url)

chowder = bs(chow.text)

scores = chowder.find_all('span', itemprop = 'ratingValue')

print(f'{len(scores)} scores found')

for score in scores:
    print(score.text)

8 scores found
88
87
87
86
85
84
83
82


In [9]:
#get whiskey name now

names = chowder.find_all('h1', itemprop = 'name')

print(f'{len(scores)} names found')

for name in names:
    print(name.text)

8 names found
Spirit Hound Single Barrel Straight Malt (Barrel 66), 45%
Dogfish Head Straight Malt, 40%
Balcones Texas Bock Straight Malt Whiskey, 50%
Woodford Reserve Straight Malt Whiskey, 45.2%
Spirit Hound Colorado Honey Malt (Barrel No. 4), 45%
Parker’s Heritage Collection Kentucky Straight Malt Whiskey 8 year old, 54%
Woodford Reserve Master’s Collection Straight Malt, 45.2%
Woodford Reserve Master’s Collection Classic Malt, 45.2%


In [10]:
#get the whiskey price now

price_int = chowder.find_all('span', itemprop = 'price')

curr_unit = chowder.find_all('span', itemprop = 'priceCurrency')

#note, the relevent information is stored in the 'content' field in both these cases
for curr in curr_unit:
    print(curr['content'])

USD
USD
USD
USD
USD
USD
USD
USD


In [11]:
#get category designation

cat_des = chowder.find_all('span', itemprop = 'category')

for cat in cat_des:
    print(cat.text)

American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey
American Malt Whiskey


In [12]:
#get blurb

blurb = chowder.find_all('div', class_ = 'review-text', itemprop = 'description')

print(f'{len(blurb)} blurbs found')

print(blurb[0].text)

8 blurbs found

Floral with some hoppy aromas to start, as well as lemon curd, vanilla, banana Runts, ripe pears, and amaretto. On the palate, gentle oakiness underpins banana pudding, whipped cream, spiced plum, amaretto, ginger candy, peaches, and chai spices of cardamom, clove, and cinnamon. Well-structured and delicate—add water sparingly.


In [13]:
#get reviewer

reviewer = chowder.find_all('span', itemprop = 'author')

for re in reviewer:
    print(re.text)

Susannah Skiver Barton
Susannah Skiver Barton
David Fleming
Jeffery Lindenmuth
Susannah Skiver Barton
John Hansell
Lew Bryson
Lew Bryson


# Write the scrape_whisky Function

In [141]:
#the Scrape_whisky function is for iterating through all the differrent style options to compile a full dataset
#the function should loop through all the styles of whisky

import time

def scrape_whisky(idArray):
    
    #set up lists for data from the site
    whis_name = []
    w_scores = []
    price_val = []
    price_curr = []
    w_cat = []
    reviews = []
    authors = []
    counter = 0
    
    #this for loop will be to loop through all the style designations
    
    for ids in idArray:
        #set up url
        loop_url = f'https://www.whiskyadvocate.com/ratings-reviews/?search=&submit=&brand_id=0&rating=0&price=0&category=0&styles_id={ids}&issue_id=0'
        
        #make the soup object
        bisque = requests.get(loop_url)
        lobster_bisque = bs(bisque.text)
        
        #scrape the data and loop through the scrapes, appending the relevant portions for each 
        nms = lobster_bisque.find_all('h1', itemprop = 'name')
        
        for n in nms:
            whis_name.append(n.text)
            
        scrs = lobster_bisque.find_all('span', itemprop = 'ratingValue')
        
        for s in scrs:
            w_scores.append(s.text)
        
        prc = lobster_bisque.find_all('span', itemprop = 'price')
        
        for p in prc:
            price_val.append(p.text)
        
        crncy = lobster_bisque.find_all('span', itemprop = 'priceCurrency')
        
        for c in crncy:
            price_curr.append(c['content'])
        
        ctgry = lobster_bisque.find_all('span', itemprop = 'category')
        
        for ct in ctgry:
            w_cat.append(ct.text)
        
        rvw = lobster_bisque.find_all('div', class_ = 'review-text', itemprop = 'description')
        
        for r in rvw:
            reviews.append(r.text)
        
        auts = lobster_bisque.find_all('span', itemprop = 'author')
        
        for au in auts:
            authors.append(au.text)
        
        #feedback
        counter += 1
        print(f'finished scrape {counter}')
        print(f'list lengths: {len(nms)}, {len(scrs)}, {len(prc)}, {len(crncy)}, {len(ctgry)}, {len(rvw)}, {len(auts)}')
        print(f'style id added: {ids}')
        
        #sleep for a little while to aviod slamming their site with too many requests.
        time.sleep(bisque.elapsed.total_seconds())
    
    #loop ends here: more feedback
    print('finished scraping')
    print(f'''list lengths are: Names: {len(whis_name)}, Scores: {len(w_scores)}, Prices: {len(price_val)}, 
          Currency: {len(price_curr)}, Category:{len(w_cat)}, Reviews: {len(reviews)}, Author: {len(authors)}''' )
              
    #package data into a dictionary
    whisDict = {
        'Name': whis_name, 'Score': w_scores, 'Price': price_val, 'Currency_unit': price_curr, 'Category': w_cat,
        'Review': reviews, 'Author' : authors
    }
    
    return whisDict


W_dict = scrape_whisky(w_ids)    


finished scrape 1
list lengths: 8, 8, 8, 8, 8, 8, 8
style id added: 68
finished scrape 2
list lengths: 58, 58, 58, 58, 58, 58, 58
style id added: 15
finished scrape 3
list lengths: 46, 46, 46, 46, 46, 46, 46
style id added: 31
finished scrape 4
list lengths: 11, 11, 11, 11, 11, 11, 11
style id added: 52
finished scrape 5
list lengths: 9, 9, 9, 9, 9, 9, 9
style id added: 27
finished scrape 6
list lengths: 1, 1, 1, 1, 1, 1, 1
style id added: 53
finished scrape 7
list lengths: 33, 33, 33, 33, 33, 33, 33
style id added: 42
finished scrape 8
list lengths: 1, 1, 1, 1, 1, 1, 1
style id added: 57
finished scrape 9
list lengths: 3, 3, 3, 3, 3, 3, 3
style id added: 55
finished scrape 10
list lengths: 9, 9, 9, 9, 9, 9, 9
style id added: 67
finished scrape 11
list lengths: 115, 115, 115, 115, 115, 115, 115
style id added: 17
finished scrape 12
list lengths: 2, 2, 2, 2, 2, 2, 2
style id added: 69
finished scrape 13
list lengths: 190, 190, 190, 190, 190, 190, 190
style id added: 3
finished scrape 14

In [173]:
#try encoding the reviews in UTF-8 to fix character display bug
#the important part of this is the utf-8-sig part the website was encoded in utf-8 and that's what bs4 and python were using
#but excel was apparently having a hard time with that detal.


dbg = W_dict['Review'].copy()

dbg = [r.strip('\n') for r in dbg]


dbg_dict = {'text' : dbg}

dbg_df = pd.DataFrame(dbg_dict)



#dbg_df.iloc[0].to_csv('desktop/independant_projects/whiskey_data_scraper/test_rev.csv', index = False, encoding = 'utf-8-sig')

In [174]:
Whis_df = pd.DataFrame(W_dict)

Whis_df.head()

,Name,Score,Price,Currency_unit,Category,Review,Author
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55,USD,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton
1,"Dogfish Head Straight Malt, 40%",87,40,USD,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40,USD,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35,USD,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65,USD,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton


In [175]:
#write out the dataset as a csv for a convenient save point

#Whis_df.to_csv('desktop/independant_projects/whiskey_data_scraper/rough_whiskey_data.csv')

In [176]:
print(W_dict['Review'][0])


Floral with some hoppy aromas to start, as well as lemon curd, vanilla, banana Runts, ripe pears, and amaretto. On the palate, gentle oakiness underpins banana pudding, whipped cream, spiced plum, amaretto, ginger candy, peaches, and chai spices of cardamom, clove, and cinnamon. Well-structured and delicate—add water sparingly.


In [177]:
#groupby currency to see if there is anything other than USD

currency_chck = Whis_df.groupby('Currency_unit').count()

currency_chck

,Name,Score,Price,Category,Review,Author
Currency_unit,,,,,,
USD,5905,5905,5905,5905,5905,5905


The Currency_unit column can be dropped: its all USD

In [178]:
Whis_df = Whis_df.drop(columns = ['Currency_unit'])

Whis_df.head()

,Name,Score,Price,Category,Review,Author
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton
1,"Dogfish Head Straight Malt, 40%",87,40,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton


In [179]:
Whis_df = Whis_df.rename(columns = {'Price': 'Price_USD'})

Whis_df.head()

,Name,Score,Price_USD,Category,Review,Author
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton
1,"Dogfish Head Straight Malt, 40%",87,40,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton


In [180]:
Whis_df.dtypes

Name         object
Score        object
Price_USD    object
Category     object
Review       object
Author       object
dtype: object

In [181]:
Whis_df['Score'] = pd.to_numeric(Whis_df['Score'])

Whis_df.dtypes

Name         object
Score         int64
Price_USD    object
Category     object
Review       object
Author       object
dtype: object

In [182]:
#Grouping by Price, to check for problem values since I couldn't retype price

prc_chk = Whis_df.groupby('Price_USD').count()

prc_chk

,Name,Score,Category,Review,Author
Price_USD,,,,,
$130,1,1,1,1,1
"1,000",2,2,2,2,2
"1,008",1,1,1,1,1
"1,052",1,1,1,1,1
"1,058",1,1,1,1,1
...,...,...,...,...,...
98.00,3,3,3,3,3
989,2,2,2,2,2
99,39,39,39,39,39


Things to do with the price column:

- Remove dollar signs and commas.

- split off a new column using / as a delimiter - This is being used in cases when the amount of whisky isn't 750 ml.

- numericize the column.

When the new column is made I can replace NaN values with 750 and drop whiskeys sold as a set, or do something else with them. 
The name column also needs to be split to remove alcohol percentage to a new column. I will also have to put up with NaN values 
in that column. 

In [183]:
#get rid of $ sign and comma

Whis_df['Price_USD'] = Whis_df['Price_USD'].str.replace('$', '')
Whis_df['Price_USD'] = Whis_df['Price_USD'].str.replace(',', '')
prc_chk = Whis_df.groupby('Price_USD').count()

prc_chk

,Name,Score,Category,Review,Author
Price_USD,,,,,
10,7,7,7,7,7
10.00,1,1,1,1,1
100,184,184,184,184,184
100.00,41,41,41,41,41
100/375ml.,1,1,1,1,1
...,...,...,...,...,...
98.00,3,3,3,3,3
989,2,2,2,2,2
99,39,39,39,39,39


In [184]:
#split the price column with / as the delimiter

pr_list = Whis_df['Price_USD'].to_list()
clean_price = []
volume = []

for pr in pr_list:
    split_list = pr.split('/')
    clean_price.append(split_list[0])
    try:
        volume.append(split_list[1])
    except:
        volume.append(750)

Whis_df['Price_USD'] = clean_price

Whis_df['Sale_volume'] = volume

Whis_df.head()
    

,Name,Score,Price_USD,Category,Review,Author,Sale_volume
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750
1,"Dogfish Head Straight Malt, 40%",87,40,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750


In [185]:
#I tried to set the value to numeric, it tripped up on line 3527
# I can't use this because the price is uncertain
Whis_df.iloc[3527]

Name                        The Balvenie 1973 43 year old, 46.6%
Score                                                         95
Price_USD                                         15000 or 60000
Category                                      Single Malt Scotch
Review         \nThis expression was matured in a European oa...
Author                                               Gavin Smith
Sale_volume                                                  set
Name: 3527, dtype: object

In [186]:
#drop this row and confirm the drop
Whis_df = Whis_df.drop([3527])

Whis_df.iloc[3527]

Name                           Glenglassaugh, 40 year old, 44.6%
Score                                                         95
Price_USD                                                2525.00
Category                                      Single Malt Scotch
Review         \nAn excellent example of an ultra-mature, she...
Author                                              John Hansell
Sale_volume                                                  750
Name: 3528, dtype: object

In [187]:
#retry the datatype reassignment

Whis_df['Price_USD'] = pd.to_numeric(Whis_df['Price_USD'])

Whis_df.dtypes

Name            object
Score            int64
Price_USD      float64
Category        object
Review          object
Author          object
Sale_volume     object
dtype: object

In [188]:
# Split out ABV
# In cases where ABV is unknown, I will assume it is 40% which is the modal value for hard liquor

name_list = Whis_df['Name'].to_list()

whis_names = []
abv = []

for name in name_list:
    nl = name.split(',')
    whis_names.append(nl[0])
    try:
        abv.append(nl[-1].strip())
    except:
        abv.append(40)

#Whis_df['Name'] = whis_names

Whis_df['Percent_ABV'] = abv

Whis_df.head()

,Name,Score,Price_USD,Category,Review,Author,Sale_volume,Percent_ABV
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45%
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40%
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50%
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2%
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45%


In [189]:
#Get rid of the percent signs in the ABV column

Whis_df['Percent_ABV'] = Whis_df['Percent_ABV'].str.replace('%', '')

Whis_df.head()

,Name,Score,Price_USD,Category,Review,Author,Sale_volume,Percent_ABV
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45


Basic procedure from here and below: I find an error where the last comma doesn't cleanly lead to a number representing ABV, and I make a change to the value. I run the iloc value first and then insert the change above it to save space. A lot of the time these are quotation marks getting pulled into the ABV value becuase of illogical English punctuation rules.

# Cleaning the ABV column

Line by line cleaning of broken ABV values

In [190]:
#Set Percent_ABV to a numeric datatype - didn't work
#check out position 55

Whis_df.iloc[55]

Name           Finger Lakes Distilling McKenzie Pure Pot Stil...
Score                                                         83
Price_USD                                                     43
Category                                        American Whiskey
Review         \n\nMade from unmalted barley, malted barley, ...
Author                                               Ted Simmons
Sale_volume                                                  750
Percent_ABV    Finger Lakes Distilling McKenzie Pure Pot Stil...
Name: 55, dtype: object

In [191]:
#Rename by concatenating, and set ABV to default value of 40%

Whis_df.iloc[55, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[55]

Name           Finger Lakes Distilling McKenzie Pure Pot Stil...
Score                                                         83
Price_USD                                                     43
Category                                        American Whiskey
Review         \n\nMade from unmalted barley, malted barley, ...
Author                                               Ted Simmons
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 55, dtype: object

In [192]:
#Set Percent_ABV to a numeric datatype - didn't work
#check out position 133

Whis_df.iloc[133, Whis_df.columns.get_loc('Percent_ABV')] = 60.2

Whis_df.iloc[133]

Name                     Little Book Chapter 1 “The Easy,” 60.2%
Score                                                         92
Price_USD                                                     80
Category                                Blended American Whiskey
Review         \nBy focusing exclusively on high-quality stra...
Author                                              Fred Minnick
Sale_volume                                                  750
Percent_ABV                                                 60.2
Name: 133, dtype: object

In [193]:
Whis_df.iloc[135, Whis_df.columns.get_loc('Percent_ABV')] = 60.2

Whis_df.iloc[135]

Name                     Little Book Chapter 1 "The Easy," 60.2%
Score                                                         91
Price_USD                                                     80
Category                                Blended American Whiskey
Review         \nIt opens with dark chocolate, coffee, hazeln...
Author                                              Fred Minnick
Sale_volume                                                  750
Percent_ABV                                                 60.2
Name: 135, dtype: object

In [194]:
Whis_df.iloc[174, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[174]

Name                                Compass Box Hedonism Maximus
Score                                                         90
Price_USD                                                    300
Category                             Blended Grain Scotch Whisky
Review         \nAn older, more exotic expression of Hedonism...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 174, dtype: object

In [195]:
Whis_df.iloc[790, Whis_df.columns.get_loc('Percent_ABV')] = 65.4
Whis_df.iloc[790]

Name            Booker’s 2014-01 “25th Anniversary Batch,” 65.4%
Score                                                         96
Price_USD                                                    100
Category                                       Bourbon/Tennessee
Review         \nThe complete package: uncut, unfiltered, ful...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 65.4
Name: 790, dtype: object

In [196]:
Whis_df.iloc[795, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[795]

Name                                            Pappy Van Winkle
Score                                                         96
Price_USD                                                    220
Category                                       Bourbon/Tennessee
Review         \nMy review of this whiskey a few years back i...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 795, dtype: object

In [197]:
Whis_df.iloc[805, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[805]

Name                                         Old Rip Van Winkle 
Score                                                         95
Price_USD                                                    350
Category                                       Bourbon/Tennessee
Review         \nDistilled in 1986. An excellent old wheated ...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 805, dtype: object

In [198]:
Whis_df.iloc[814, Whis_df.columns.get_loc('Percent_ABV')] = 45

Whis_df.iloc[814]

Name           Buffalo Trace Experimental Collection, 1995 Vi...
Score                                                         95
Price_USD                                                     47
Category                                       Bourbon/Tennessee
Review         \nSurprisingly light and fresh for a 15 year o...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   45
Name: 814, dtype: object

In [199]:
Whis_df.iloc[856, Whis_df.columns.get_loc('Percent_ABV')] = 60.20

Whis_df.iloc[856]

Name           Bardstown Bourbon Co. Copper & Kings Apple Bra...
Score                                                         94
Price_USD                                                    125
Category                                       Bourbon/Tennessee
Review         \nAromas of corn meal, corn muffins, and butte...
Author                                             David Fleming
Sale_volume                                                  750
Percent_ABV                                                 60.2
Name: 856, dtype: object

In [200]:
Whis_df.iloc[952, Whis_df.columns.get_loc('Percent_ABV')] = 64.4

Whis_df.iloc[952]

Name                      Booker’s 2018-02 “Backyard BBQ,” 64.4%
Score                                                         93
Price_USD                                                     75
Category                                       Bourbon/Tennessee
Review         \nTypical of Booker’s this bourbon shows lots ...
Author                                        Jeffery Lindenmuth
Sale_volume                                                  750
Percent_ABV                                                 64.4
Name: 952, dtype: object

In [201]:
Whis_df.iloc[969, Whis_df.columns.get_loc('Percent_ABV')] = 63.35

Whis_df.iloc[969]

Name                    Booker’s 2018-03 “Kentucky Chew,” 63.35%
Score                                                         93
Price_USD                                                     70
Category                                       Bourbon/Tennessee
Review         \nOriginally touted by Booker Noe, the “Kentuc...
Author                                    Susannah Skiver Barton
Sale_volume                                                  750
Percent_ABV                                                63.35
Name: 969, dtype: object

In [202]:
Whis_df.iloc[978, Whis_df.columns.get_loc('Percent_ABV')] = 61.3

Whis_df.iloc[978]

Name                Little Book Chapter 3 “The Road Home,” 61.3%
Score                                                         93
Price_USD                                                    125
Category                                       Bourbon/Tennessee
Review         \nThe chewy caramel aroma is bursting with all...
Author                                        Jeffery Lindenmuth
Sale_volume                                                  750
Percent_ABV                                                 61.3
Name: 978, dtype: object

In [203]:
Whis_df.iloc[991, Whis_df.columns.get_loc('Percent_ABV')] = 63.7

Whis_df.iloc[991]

Name                  Booker’s 2018-01 "Kathleen’s Batch," 63.7%
Score                                                         92
Price_USD                                                     70
Category                                       Bourbon/Tennessee
Review         \nBright and pronounced caramel in many forms,...
Author                                              Fred Minnick
Sale_volume                                                  750
Percent_ABV                                                 63.7
Name: 991, dtype: object

In [204]:
Whis_df.iloc[1022, Whis_df.columns.get_loc('Percent_ABV')] = 64.25

Whis_df.iloc[1022]

Name                    Booker’s 2017-01 “Tommy’s Batch,” 64.25%
Score                                                         92
Price_USD                                                     70
Category                                       Bourbon/Tennessee
Review         \nAt 6 years, 4 months, and 6 days, this is be...
Author                                              Fred Minnick
Sale_volume                                                  750
Percent_ABV                                                64.25
Name: 1022, dtype: object

In [205]:
Whis_df.iloc[1038, Whis_df.columns.get_loc('Percent_ABV')] = 64.05

Whis_df.iloc[1038]

Name                       Booker’s 2017-04 “Sip Awhile,” 64.05%
Score                                                         92
Price_USD                                                     73
Category                                       Bourbon/Tennessee
Review         \nRosewater, almond extract, dark cherries, ro...
Author                                              Fred Minnick
Sale_volume                                                  750
Percent_ABV                                                64.05
Name: 1038, dtype: object

In [206]:
Whis_df.iloc[1041, Whis_df.columns.get_loc('Percent_ABV')] = 62.95

Whis_df.iloc[1041]

Name                   Booker’s 2019-01 “Teresa’s Batch,” 62.95%
Score                                                         92
Price_USD                                                     80
Category                                       Bourbon/Tennessee
Review         \nOn the nose, there’s Golden Grahams, iced te...
Author                                    Susannah Skiver Barton
Sale_volume                                                  750
Percent_ABV                                                62.95
Name: 1041, dtype: object

In [207]:
Whis_df.iloc[1052, Whis_df.columns.get_loc('Percent_ABV')] = 64

Whis_df.iloc[1052]

Name                       Booker’s 2018-04 “Kitchen Table,” 64%
Score                                                         92
Price_USD                                                     75
Category                                       Bourbon/Tennessee
Review         \nA classic bourbon nose, as butterscotch hard...
Author                                        Jeffery Lindenmuth
Sale_volume                                                  750
Percent_ABV                                                   64
Name: 1052, dtype: object

In [208]:
Whis_df.iloc[1093, Whis_df.columns.get_loc('Percent_ABV')] = 63.95

Whis_df.iloc[1093]

Name                  Booker’s 2016-01 “Bluegrass Batch,” 63.95%
Score                                                         91
Price_USD                                                     60
Category                                       Bourbon/Tennessee
Review         \nIf Picasso sketched bourbon, it would look l...
Author                                              Fred Minnick
Sale_volume                                                  750
Percent_ABV                                                63.95
Name: 1093, dtype: object

In [209]:
Whis_df.iloc[1225, Whis_df.columns.get_loc('Percent_ABV')] = 43.3

Whis_df.iloc[1225]

Name           Evan Williams Single Barrel 1994 Vintage (Barr...
Score                                                         89
Price_USD                                                     25
Category                                       Bourbon/Tennessee
Review         \nThis is an elegant whiskey with lots of fine...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 43.3
Name: 1225, dtype: object

In [210]:
Whis_df.iloc[1290, Whis_df.columns.get_loc('Percent_ABV')] = 46.2

Whis_df.iloc[1290]

Name           Woodford Reserve Master’s Collection “New Cask...
Score                                                         88
Price_USD                                                    100
Category                                       Bourbon/Tennessee
Review         \nA set of two 100% rye whiskeys, triple disti...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 46.2
Name: 1290, dtype: object

In [211]:
Whis_df.iloc[1294, Whis_df.columns.get_loc('Percent_ABV')] = 50.2

Whis_df.iloc[1294]

Name           Woodford Reserve Master's Collection 'Seasoned...
Score                                                         88
Price_USD                                                     90
Category                                       Bourbon/Tennessee
Review         \nThe fourth in the series of limited-edition ...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 50.2
Name: 1294, dtype: object

In [212]:
Whis_df.iloc[1294, Whis_df.columns.get_loc('Percent_ABV')] = 45

Whis_df.iloc[1294]

Name           Woodford Reserve Master's Collection 'Seasoned...
Score                                                         88
Price_USD                                                     90
Category                                       Bourbon/Tennessee
Review         \nThe fourth in the series of limited-edition ...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   45
Name: 1294, dtype: object

In [213]:
Whis_df.iloc[1327, Whis_df.columns.get_loc('Percent_ABV')] = 45

Whis_df.iloc[1327]

Name           Buffalo Trace Experimental Collection, 1995 Vi...
Score                                                         88
Price_USD                                                     47
Category                                       Bourbon/Tennessee
Review         \nDark, big, and brooding. A mouthful of oak s...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   45
Name: 1327, dtype: object

In [214]:
Whis_df.iloc[1332, Whis_df.columns.get_loc('Percent_ABV')] = 64.35

Whis_df.iloc[1332]

Name             Booker’s 2015-01 “Big Man, Small Batch,” 64.35%
Score                                                         88
Price_USD                                                     55
Category                                       Bourbon/Tennessee
Review         \nThe first in a series of limited edition rel...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                64.35
Name: 1332, dtype: object

In [215]:
Whis_df.iloc[1364, Whis_df.columns.get_loc('Percent_ABV')] = 58

Whis_df.iloc[1364]

Name           Four Roses Mariage Collection (2009 Release), ...
Score                                                         87
Price_USD                                                     80
Category                                       Bourbon/Tennessee
Review         \nA combination of 10 and 19 year old bourbons...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   58
Name: 1364, dtype: object

In [216]:
Whis_df.iloc[1368, Whis_df.columns.get_loc('Percent_ABV')] = 46.2

Whis_df.iloc[1368]

Name           Woodford Reserve Master’s Collection “Aged Cas...
Score                                                         87
Price_USD                                                    100
Category                                       Bourbon/Tennessee
Review         \nA set of two 100% rye whiskeys, triple disti...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 46.2
Name: 1368, dtype: object

In [217]:
Whis_df.iloc[1403, Whis_df.columns.get_loc('Percent_ABV')] = 63.5

Whis_df.iloc[1403]

Name                 Booker’s 2015-04 “Oven Buster Batch,” 63.5%
Score                                                         87
Price_USD                                                     60
Category                                       Bourbon/Tennessee
Review         \nAged over 6 years, this is a rather feminine...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 63.5
Name: 1403, dtype: object

In [218]:
Whis_df.iloc[1438, Whis_df.columns.get_loc('Percent_ABV')] = 63.7

Whis_df.iloc[1438]

Name                Booker’s 2017-02 “Blue Knights Batch,” 63.7%
Score                                                         86
Price_USD                                                     75
Category                                       Bourbon/Tennessee
Review         \nThink corn: roasted, canned, creamed, mashed...
Author                                              Fred Minnick
Sale_volume                                                  750
Percent_ABV                                                 63.7
Name: 1438, dtype: object

In [219]:
Whis_df.iloc[1561, Whis_df.columns.get_loc('Percent_ABV')] = 45

Whis_df.iloc[1561]

Name                                 Jim Beam “Devil’s Cut,” 45%
Score                                                         83
Price_USD                                                     24
Category                                       Bourbon/Tennessee
Review         \nMore wood influence than the standard Jim Be...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   45
Name: 1561, dtype: object

In [220]:
Whis_df.iloc[1590, Whis_df.columns.get_loc('Percent_ABV')] = 64.5

Whis_df.iloc[1590]

Name               Booker’s 2016-03 “Toogie’s Invitation,” 64.5%
Score                                                         82
Price_USD                                                     60
Category                                       Bourbon/Tennessee
Review         \nAt 6 years, 4 months, and 4 days old, there’...
Author                                              Fred Minnick
Sale_volume                                                  750
Percent_ABV                                                 64.5
Name: 1590, dtype: object

In [221]:
Whis_df.iloc[1783, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[1783]

Name                          Black Velvet Onyx 12 year old. 40%
Score                                                         88
Price_USD                                                     28
Category                                                Canadian
Review         \nDry and mildly dusty on first nosing, with a...
Author                                      Davin de Kergommeaux
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 1783, dtype: object

In [222]:
Whis_df.iloc[1828, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[1828]

Name                           Canadian Club Classic 12, 40% ABV
Score                                                         85
Price_USD                                                     22
Category                                                Canadian
Review         \nNose is much more integrated than the others...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 1828, dtype: object

In [223]:
Whis_df.iloc[1841, Whis_df.columns.get_loc('Percent_ABV')] = 62.2

Whis_df.iloc[1841]

Name                                  Glen Breton Ice, 62.2% ABV
Score                                                         84
Price_USD                                                     50
Category                                                Canadian
Review         \nAged 10 years in icewine barrels — a Canadia...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                 62.2
Name: 1841, dtype: object

In [224]:
Whis_df.iloc[1872, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[1872]

Name                              Canadian Club Reserve, 40% ABV
Score                                                         82
Price_USD                                                     17
Category                                                Canadian
Review         \nA bit of heat, faint nose of putty, light br...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 1872, dtype: object

In [225]:
Whis_df.iloc[1873, Whis_df.columns.get_loc('Percent_ABV')] = 41.3

Whis_df.iloc[1873]

Name                        Canadian Club Sherry Cask, 41.3% ABV
Score                                                         82
Price_USD                                                     28
Category                                                Canadian
Review         \n8 years old and sherry finished, and it show...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                 41.3
Name: 1873, dtype: object

In [226]:
Whis_df.iloc[1892, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[1892]

Name                                          Danfield's Private
Score                                                         81
Price_USD                                                     22
Category                                                Canadian
Review         \nLight in body and on the nose, but clean and...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 1892, dtype: object

In [227]:
Whis_df.iloc[1899, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[1899]

Name                               Canadian Club Whisky, 40% ABV
Score                                                         80
Price_USD                                                     14
Category                                                Canadian
Review         \nPale, almost peach color. Light aromas of ca...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 1899, dtype: object

In [228]:
Whis_df.iloc[1921, Whis_df.columns.get_loc('Percent_ABV')] = 50

Whis_df.iloc[1921]

Name            Canadian Club 100 Proof Export Strength, 50% ABV
Score                                                         77
Price_USD                                                     20
Category                                                Canadian
Review         \nHot solvents on the nose, reminiscent of Hin...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                   50
Name: 1921, dtype: object

In [229]:
Whis_df.iloc[2161, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[2161]

Name                 Wyoming Double Cask Sherry Finished Bourbon
Score                                                         83
Price_USD                                                     60
Category                                           Craft Whiskey
Review         \nPale gold in color. Generous oak imparts hon...
Author                                        Jeffery Lindenmuth
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 2161, dtype: object

In [230]:
Whis_df.iloc[2364, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[2364]

Name                                                    JP Wiser
Score                                                         81
Price_USD                                                     21
Category                              Flavored Whisky & Liqueurs
Review         \nBurnt caramel, sweet—almost candied—but not ...
Author                                      Davin de Kergommeaux
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 2364, dtype: object

In [231]:
Whis_df.iloc[2409, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[2409]

Name                                           Kornog Toal Esa Z
Score                                                         90
Price_USD                                                     77
Category                                           French Whisky
Review         \nKornog is the peated version of Glann ar Mor...
Author                                           Dominic Roskrow
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 2409, dtype: object

In [232]:
Whis_df.iloc[2420, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[2420]

Name                                              Kornog Taouarc
Score                                                         88
Price_USD                                                     77
Category                                           French Whisky
Review         \nOf all the releases I've tasted from this di...
Author                                           Dominic Roskrow
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 2420, dtype: object

In [233]:
Whis_df.iloc[2479, Whis_df.columns.get_loc('Percent_ABV')] = 50

Whis_df.iloc[2479]

Name                                         Amrut 'Fusion,' 50%
Score                                                         89
Price_USD                                                     50
Category                                           Indian Whisky
Review         \nBalance, complexity, and surprising maturity...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   50
Name: 2479, dtype: object

In [234]:
Whis_df.iloc[2778, Whis_df.columns.get_loc('Percent_ABV')] = 50

Whis_df.iloc[2778]

Name                                        Fuji-Sanroku 50° 50%
Score                                                         92
Price_USD                                                     53
Category                                         Japanese Whisky
Review         \nJota Tanaka, Kirin’s chief blender, utilizes...
Author                                           Jonny McCormick
Sale_volume                                                  750
Percent_ABV                                                   50
Name: 2778, dtype: object

In [235]:
Whis_df.iloc[2791, Whis_df.columns.get_loc('Percent_ABV')] = 43

Whis_df.iloc[2791]

Name                           Mars The Lucky Cat “Ash ’99,” 43%
Score                                                         91
Price_USD                                                    100
Category                                         Japanese Whisky
Review         \nYes, there is a cute, mewing black kitten on...
Author                                           Jonny McCormick
Sale_volume                                                  750
Percent_ABV                                                   43
Name: 2791, dtype: object

In [236]:
Whis_df.iloc[2794, Whis_df.columns.get_loc('Percent_ABV')] = 58

Whis_df.iloc[2794]

Name           Ichiro’s Malt, Hanyu 23 year old “sherry casks...
Score                                                         91
Price_USD                                                    695
Category                                         Japanese Whisky
Review         \nThis oldie (from a distillery which closed i...
Author                                                Dave Broom
Sale_volume                                                  750
Percent_ABV                                                   58
Name: 2794, dtype: object

In [237]:
Whis_df.iloc[2828, Whis_df.columns.get_loc('Percent_ABV')] = 60.7

Whis_df.iloc[2828]

Name           Karuizawa cask 4592, 1977 (bottle 2010), ‘Noh ...
Score                                                         89
Price_USD                                                    200
Category                                         Japanese Whisky
Review         \nThe (sadly mothballed) Karuizawa distillery ...
Author                                                Dave Broom
Sale_volume                                                  750
Percent_ABV                                                 60.7
Name: 2828, dtype: object

In [238]:
Whis_df.iloc[2902, Whis_df.columns.get_loc('Percent_ABV')] = 59.4

Whis_df.iloc[2902]

Name              Little Book Chapter 2 “Noe Simple Task,” 59.4%
Score                                                         93
Price_USD                                                    100
Category                                           Miscellaneous
Review         \nA fascinating flurry of aromas—red fruit pun...
Author                                        Jeffery Lindenmuth
Sale_volume                                                  750
Percent_ABV                                                 59.4
Name: 2902, dtype: object

In [239]:
#row 2903 is a dupe

Whis_df = Whis_df.drop([2903])

Whis_df.iloc[2903]

Name                         Barrell Whiskey (Batch 001), 61.25%
Score                                                         92
Price_USD                                                     59
Category                                           Miscellaneous
Review         \n7 year old whiskey (an unspecified “corn, ry...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                61.25
Name: 2904, dtype: object

In [240]:
Whis_df.iloc[2952, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[2952]

Name                                                 New Zealand
Score                                                         88
Price_USD                                                    233
Category                                             New Zealand
Review         \nRugby union fans will recognize the name and...
Author                                           Dominic Roskrow
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 2953, dtype: object

In [241]:
Whis_df.iloc[2960, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[2960]

Name                                         Thomson 17 year old
Score                                                         79
Price_USD                                                     95
Category                                             New Zealand
Review         \nThomson has discovered and bottled stocks fr...
Author                                           Dominic Roskrow
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 2961, dtype: object

In [242]:
Whis_df.iloc[3131, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[3131]

Name                         Minor Case Sherry Cask Finished Rye
Score                                                         87
Price_USD                                                     50
Category                                             Rye Whiskey
Review         \nThe sherry cask finish asserts itself right ...
Author                                    Susannah Skiver Barton
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 3132, dtype: object

In [243]:
Whis_df.iloc[3225, Whis_df.columns.get_loc('Percent_ABV')] = 46.5

Whis_df.iloc[3225]

Name                          Grand Traverse "Ole George," 46.5%
Score                                                         80
Price_USD                                                     48
Category                                             Rye Whiskey
Review         \nSweet (caramel, butterscotch), botanical, an...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 46.5
Name: 3226, dtype: object

In [244]:
Whis_df.iloc[3437, Whis_df.columns.get_loc('Percent_ABV')] = 46.5

Whis_df.iloc[3437]

Name           Woodstone Creek Single Malt “The Murray Barrel...
Score                                                         83
Price_USD                                                    118
Category                            Single Malt American Whiskey
Review         \nNice color for a 10 year old. Puffing peat o...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                 46.5
Name: 3438, dtype: object

In [245]:
Whis_df.iloc[3442, Whis_df.columns.get_loc('Percent_ABV')] = 44.4

Whis_df.iloc[3442]

Name                                        The Notch, 44.4% ABV
Score                                                         82
Price_USD                                                    888
Category                            Single Malt American Whiskey
Review         \nThe price, the proof, and 8 years of age ref...
Author                                                Lew Bryson
Sale_volume                                                  750
Percent_ABV                                                 44.4
Name: 3443, dtype: object

In [246]:
Whis_df.iloc[3485, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[3485]

Name                          St. George’s Chapter 6, 3 year old
Score                                                         80
Price_USD                                                     59
Category                              Single Malt English Whisky
Review         \nEngland’s sole whisky distillery (currently,...
Author                                                Dave Broom
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 3486, dtype: object

In [247]:
Whis_df.iloc[3669, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[3669]

Name           The Balvenie 30 year old 1973 Vintage (Cask #9...
Score                                                         93
Price_USD                                                    400
Category                                      Single Malt Scotch
Review         \nThe last Balvenie Vintage whisky I tasted th...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 3671, dtype: object

In [248]:
Whis_df.iloc[3677, Whis_df.columns.get_loc('Percent_ABV')] = 46.7

Whis_df.iloc[3677]

Name                         The Glenrothes “John Ramsay,” 46.7%
Score                                                         93
Price_USD                                                   1000
Category                                      Single Malt Scotch
Review         \nMade from whisky aged in second fill America...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 46.7
Name: 3679, dtype: object

In [249]:
Whis_df.iloc[3731, Whis_df.columns.get_loc('Percent_ABV')] = 46

Whis_df.iloc[3731]

Name                          Kilchoman 'Inaugural Release,' 46%
Score                                                         92
Price_USD                                                     60
Category                                      Single Malt Scotch
Review         \nAged 2.5 to 3 years on bourbon casks and the...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   46
Name: 3733, dtype: object

In [250]:
Whis_df.iloc[3731, Whis_df.columns.get_loc('Percent_ABV')] = 46

Whis_df.iloc[3731]

Name                          Kilchoman 'Inaugural Release,' 46%
Score                                                         92
Price_USD                                                     60
Category                                      Single Malt Scotch
Review         \nAged 2.5 to 3 years on bourbon casks and the...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   46
Name: 3733, dtype: object

In [251]:
Whis_df.iloc[3774, Whis_df.columns.get_loc('Percent_ABV')] = 55

Whis_df.iloc[3774]

Name                           Highland Park “Saint Magnus,” 55%
Score                                                         92
Price_USD                                                    132
Category                                      Single Malt Scotch
Review         \nThe second in a series of three high-strengt...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   55
Name: 3776, dtype: object

In [252]:
Whis_df.iloc[4263, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[4263]

Name                                  Glenfarclas £511. 19s. 0d.
Score                                                         89
Price_USD                                                    130
Category                                      Single Malt Scotch
Review         \nUnusually named—it refers to the price paid ...
Author                                                Dave Broom
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 4265, dtype: object

In [253]:
Whis_df.iloc[4277, Whis_df.columns.get_loc('Percent_ABV')] = 46

Whis_df.iloc[4277]

Name                         Old Pulteney WK209 ‘Good Hope,’ 46%
Score                                                         89
Price_USD                                                     63
Category                                      Single Malt Scotch
Review         \nWK209 is named after a steam herring drifter...
Author                                               Gavin Smith
Sale_volume                                                  750
Percent_ABV                                                   46
Name: 4279, dtype: object

In [254]:
Whis_df.iloc[4316, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[4316]

Name                                             Murray McDavid 
Score                                                         89
Price_USD                                                    150
Category                                      Single Malt Scotch
Review         \nYellow gold color. Fresh, floral aromas (rem...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 4318, dtype: object

In [255]:
Whis_df.iloc[4366, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[4366]

Name                                                   Cadenhead
Score                                                         89
Price_USD                                                    125
Category                                      Single Malt Scotch
Review         \nAntique gold color. Aromas of peat smoke, da...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 4368, dtype: object

In [256]:
Whis_df.iloc[4389, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[4389]

Name           Wemyss Malts ‘Honey Spice’ (distilled at Bunna...
Score                                                         88
Price_USD                                                     96
Category                                      Single Malt Scotch
Review         \nBunnahabhain, along with Bowmore, is Islay’s...
Author                                                Dave Broom
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 4391, dtype: object

In [257]:
Whis_df.iloc[4410, Whis_df.columns.get_loc('Percent_ABV')] = 52.3

Whis_df.iloc[4410]

Name                       Arran ‘The Devil’s Punch Bowl,’ 52.3%
Score                                                         88
Price_USD                                                    130
Category                                      Single Malt Scotch
Review         \n6,660 bottles of The Devil’s Punch Bowl have...
Author                                               Gavin Smith
Sale_volume                                                  750
Percent_ABV                                                 52.3
Name: 4412, dtype: object

In [258]:
Whis_df.iloc[4551, Whis_df.columns.get_loc('Percent_ABV')] = 45

Whis_df.iloc[4551]

Name           Blackadder Single Cask #30013 (distilled at Li...
Score                                                         88
Price_USD                                                    135
Category                                      Single Malt Scotch
Review         \nAlso known as St. Magdalene, this Lowland di...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   45
Name: 4553, dtype: object

In [259]:
Whis_df.iloc[4654, Whis_df.columns.get_loc('Percent_ABV')] = 43

Whis_df.iloc[4654]

Name                     Royal Lochnagar 'Selected Reserve,' 43%
Score                                                         87
Price_USD                                                    210
Category                                      Single Malt Scotch
Review         \nSo nice to see this whisky available here in...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   43
Name: 4656, dtype: object

In [260]:
Whis_df.iloc[4662, Whis_df.columns.get_loc('Percent_ABV')] = 43

Whis_df.iloc[4662]

Name                              Glenrothes, 1994 Vintage, $43%
Score                                                         87
Price_USD                                                     78
Category                                      Single Malt Scotch
Review         \nVery bright and lively, with a nice balance ...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   43
Name: 4664, dtype: object

In [261]:
Whis_df.iloc[4667, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[4667]

Name                               Ardmore Traditional Cask 1998
Score                                                         87
Price_USD                                                     50
Category                                      Single Malt Scotch
Review         \nA single cask of Ardmore aged in a first-fil...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 4669, dtype: object

In [262]:
Whis_df.iloc[4756, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[4756]

Name           Cask & Thistle 29 year old 1974 Vintage (disti...
Score                                                         87
Price_USD                                                    110
Category                                      Single Malt Scotch
Review         \nA prime component in Dewar’s blended scotch....
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 4758, dtype: object

In [263]:
Whis_df.iloc[4771, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[4771]

Name           Wild Scotsman (distilled at Ben Nevis), 13 yea...
Score                                                         87
Price_USD                                                     70
Category                                      Single Malt Scotch
Review         \nVery robust and chewy-this whisky has plenty...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 4773, dtype: object

In [264]:
Whis_df.iloc[4987, Whis_df.columns.get_loc('Percent_ABV')] = 45

Whis_df.iloc[4987]

Name           Blackadder Single Cask #1101 (distilled at Lon...
Score                                                         86
Price_USD                                                    135
Category                                      Single Malt Scotch
Review         \nA big and sappy Longmorn, with notes of chew...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   45
Name: 4989, dtype: object

In [265]:
Whis_df.iloc[5023, Whis_df.columns.get_loc('Percent_ABV')] = 46

Whis_df.iloc[5023]

Name           Wemyss Malts (distilled at Glen Scotia) 'Straw...
Score                                                         85
Price_USD                                                    174
Category                                      Single Malt Scotch
Review         \nThis single cask variant of Glen Scotia has ...
Author                                               Gavin Smith
Sale_volume                                                  750
Percent_ABV                                                   46
Name: 5025, dtype: object

In [266]:
Whis_df.iloc[5040, Whis_df.columns.get_loc('Percent_ABV')] = 43

Whis_df.iloc[5040]

Name           Dun Bheagan (distilled at Caol Ila) 1993 Vinta...
Score                                                         85
Price_USD                                                     56
Category                                      Single Malt Scotch
Review         \nAn unusual Caol Ila, in that it was finished...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   43
Name: 5042, dtype: object

In [267]:
Whis_df.iloc[5116, Whis_df.columns.get_loc('Percent_ABV')] = 43

Whis_df.iloc[5116]

Name                       The Glenrothes, 1992 Vintage, 43% ABV
Score                                                         85
Price_USD                                                     55
Category                                      Single Malt Scotch
Review         \nA very fragrant, fruity whisky (orange, tang...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   43
Name: 5118, dtype: object

In [268]:
Whis_df.iloc[5138, Whis_df.columns.get_loc('Percent_ABV')] = 56.4

Whis_df.iloc[5138]

Name           Gordon & MacPhail Single Cask #12274 (distille...
Score                                                         85
Price_USD                                                     65
Category                                      Single Malt Scotch
Review         \nArdmore whiskies I’ve enjoyed in the past ha...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 56.4
Name: 5140, dtype: object

In [269]:
Whis_df.iloc[5140, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[5140]

Name           Duncan Taylor NC2 (distilled at Imperial) 1997...
Score                                                         85
Price_USD                                                     60
Category                                      Single Malt Scotch
Review         \nPale straw. One of a seemingly never-ending ...
Author                                                Dave Broom
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 5142, dtype: object

In [270]:
Whis_df.iloc[5154, Whis_df.columns.get_loc('Percent_ABV')] = 59.1

Whis_df.iloc[5154]

Name           Montgomerie's Single Cask #910 (distilled at B...
Score                                                         85
Price_USD                                                     65
Category                                      Single Malt Scotch
Review         \n(Exclusive to Park Ave Liquors, New York, NY...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 59.1
Name: 5156, dtype: object

In [271]:
Whis_df.iloc[5198, Whis_df.columns.get_loc('Percent_ABV')] = 57.9

Whis_df.iloc[5198]

Name           Gordon & MacPhail Single Cask #1961 (distilled...
Score                                                         84
Price_USD                                                     65
Category                                      Single Malt Scotch
Review         \nIt is unusual to find a Bruichladdich whisky...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 57.9
Name: 5200, dtype: object

In [272]:
Whis_df.iloc[5215, Whis_df.columns.get_loc('Percent_ABV')] = 46

Whis_df.iloc[5215]

Name                        Wemyss Malts ‘Caribbean Fruits,’ 46%
Score                                                         84
Price_USD                                                    127
Category                                      Single Malt Scotch
Review         \nWemyss Malts names the sources of these sing...
Author                                               Gavin Smith
Sale_volume                                                  750
Percent_ABV                                                   46
Name: 5217, dtype: object

In [273]:
Whis_df.iloc[5221, Whis_df.columns.get_loc('Percent_ABV')] = 46

Whis_df.iloc[5221]

Name                              Edradour 'Ballechin (#4),' 46%
Score                                                         84
Price_USD                                                    100
Category                                      Single Malt Scotch
Review         \nThe fourth release of peated Edradour whisky...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   46
Name: 5223, dtype: object

In [274]:
Whis_df.iloc[5359, Whis_df.columns.get_loc('Percent_ABV')] = 56.1

Whis_df.iloc[5359]

Name           Duncan Taylor (distilled at Glen Grant) 1972 V...
Score                                                         83
Price_USD                                                    155
Category                                      Single Malt Scotch
Review         \nI don’t normally like to add water to old wh...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 56.1
Name: 5361, dtype: object

In [275]:
Whis_df.iloc[5422, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[5422]

Name                                   The Macallan Whisky Maker
Score                                                         83
Price_USD                                                     90
Category                                      Single Malt Scotch
Review         \nFull gold. Oily in texture, with ripe barley...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 5424, dtype: object

In [276]:
Whis_df.iloc[5456, Whis_df.columns.get_loc('Percent_ABV')] = 43

Whis_df.iloc[5456]

Name                          The Speyside, 12 year old, 43% ABV
Score                                                         82
Price_USD                                                     30
Category                                      Single Malt Scotch
Review         \nA relative newcomer to the Scotch whisky ind...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   43
Name: 5458, dtype: object

In [277]:
Whis_df.iloc[5464, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[5464]

Name                               Tormore, 12 year old, 40% ABV
Score                                                         82
Price_USD                                                     42
Category                                      Single Malt Scotch
Review         \nSoft and creamy in texture. A delicate mélan...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 5466, dtype: object

In [278]:
Whis_df.iloc[5467, Whis_df.columns.get_loc('Percent_ABV')] = 56

Whis_df.iloc[5467]

Name           Gordon & MacPhail Single Cask #5054 (distilled...
Score                                                         82
Price_USD                                                     65
Category                                      Single Malt Scotch
Review         \nHere’s a whisky not seen very often in the U...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   56
Name: 5469, dtype: object

In [279]:
Whis_df.iloc[5479, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[5479]

Name           Gordon & MacPhail (distilled at Glenlossie), 2...
Score                                                         82
Price_USD                                                     90
Category                                      Single Malt Scotch
Review         \nFirm on the palate and quite mature. Good ba...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 5481, dtype: object

In [280]:
Whis_df.iloc[5540, Whis_df.columns.get_loc('Percent_ABV')] = 55.7

Whis_df.iloc[5540]

Name           Blackadder Raw Cask #6437 (distilled at Glen G...
Score                                                         81
Price_USD                                                    175
Category                                      Single Malt Scotch
Review         \nFrom a sherry hogshead, which is confirmed b...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 55.7
Name: 5542, dtype: object

In [281]:
Whis_df.iloc[5556, Whis_df.columns.get_loc('Percent_ABV')] = 53.6

Whis_df.iloc[5556]

Name           Hart Brothers (distilled at Glenturret), 10 ye...
Score                                                         81
Price_USD                                                     63
Category                                      Single Malt Scotch
Review         \nVery fragrant and floral, with bright notes ...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 53.6
Name: 5558, dtype: object

In [282]:
Whis_df.iloc[5560, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[5560]

Name           Berry Brothers & Rudd (distilled at Cragganmor...
Score                                                         81
Price_USD                                                     93
Category                                      Single Malt Scotch
Review         \nLight straw. Initially this is quite hot and...
Author                                                Dave Broom
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 5562, dtype: object

In [283]:
Whis_df.iloc[5578, Whis_df.columns.get_loc('Percent_ABV')] = 59.2

Whis_df.iloc[5578]

Name           Gordon & MacPhail Single Cask #1535 (distilled...
Score                                                         80
Price_USD                                                     80
Category                                      Single Malt Scotch
Review         \nA very traditional Rosebank-aged in a refill...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                 59.2
Name: 5580, dtype: object

In [284]:
Whis_df.iloc[5721, Whis_df.columns.get_loc('Percent_ABV')] = 46.1

Whis_df.iloc[5721]

Name                               Mackmyra Swedish whisky 46.1%
Score                                                         88
Price_USD                                                     55
Category                              Single Malt Swedish Whisky
Review         \nMackmyra has at long last reached the United...
Author                                           Dominic Roskrow
Sale_volume                                                  750
Percent_ABV                                                 46.1
Name: 5723, dtype: object

In [285]:
Whis_df.iloc[5805, Whis_df.columns.get_loc('Percent_ABV')] = 40

Whis_df.iloc[5805]

Name                                                 Blackadder 
Score                                                         80
Price_USD                                                     90
Category                                 Unspecified Single Malt
Review         \nVery pale gold color. Aggressive notes of ta...
Author                                              John Hansell
Sale_volume                                                  750
Percent_ABV                                                   40
Name: 5807, dtype: object

In [286]:
Whis_df['Percent_ABV'] = pd.to_numeric(Whis_df['Percent_ABV'])

Whis_df.dtypes

Name            object
Score            int64
Price_USD      float64
Category        object
Review          object
Author          object
Sale_volume     object
Percent_ABV    float64
dtype: object

In [287]:
Whis_df.head()

,Name,Score,Price_USD,Category,Review,Author,Sale_volume,Percent_ABV
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45.0
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40.0
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50.0
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45.0


In [288]:
#write out clean dataset


#Whis_df.to_csv('desktop/independant_projects/whiskey_data_scraper/clean_whiskey_data1.csv', index = False)


In [289]:
Whis_df

,Name,Score,Price_USD,Category,Review,Author,Sale_volume,Percent_ABV
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45.0
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40.0
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50.0
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45.0
...,...,...,...,...,...,...,...,...
5900,World Whiskey Society Goalong 5 year old Bourb...,86,150.0,World Whisky,\n\nThe first Chinese single malt to be export...,Susannah Skiver Barton,750,40.0
5901,"Sierra Norte Single Barrel White, 45%",86,50.0,World Whisky,\nWhite has the most familiar corn whiskey cha...,Jonny McCormick,750,45.0
5902,"Sierra Norte Single Barrel Purple Corn, 45%",85,50.0,World Whisky,\nPepper and allspice light up with savory bri...,Jonny McCormick,750,45.0
5903,That Boutique-y Whisky Company (distilled at S...,85,90.0,World Whisky,\nWhite pepper-dappled summer fruits on the no...,Jonny McCormick,750,52.5


In [290]:
Whis_df['Name'].value_counts

<bound method IndexOpsMixin.value_counts of 0       Spirit Hound Single Barrel Straight Malt (Barr...
1                         Dogfish Head Straight Malt, 40%
2          Balcones Texas Bock Straight Malt Whiskey, 50%
3           Woodford Reserve Straight Malt Whiskey, 45.2%
4       Spirit Hound Colorado Honey Malt (Barrel No. 4...
                              ...                        
5900    World Whiskey Society Goalong 5 year old Bourb...
5901                Sierra Norte Single Barrel White, 45%
5902          Sierra Norte Single Barrel Purple Corn, 45%
5903    That Boutique-y Whisky Company (distilled at S...
5904             Abasolo 100% Ancestral Corn Whiskey, 43%
Name: Name, Length: 5903, dtype: object>

In [291]:
Whis_df.drop_duplicates(inplace = True)

Whis_df

,Name,Score,Price_USD,Category,Review,Author,Sale_volume,Percent_ABV
0,Spirit Hound Single Barrel Straight Malt (Barr...,88,55.0,American Malt Whiskey,"\nFloral with some hoppy aromas to start, as w...",Susannah Skiver Barton,750,45.0
1,"Dogfish Head Straight Malt, 40%",87,40.0,American Malt Whiskey,\nLots of confectionary notes on the nose: ora...,Susannah Skiver Barton,750,40.0
2,"Balcones Texas Bock Straight Malt Whiskey, 50%",87,40.0,American Malt Whiskey,\nBalcones has partnered with fellow Texan Spo...,David Fleming,750,50.0
3,"Woodford Reserve Straight Malt Whiskey, 45.2%",86,35.0,American Malt Whiskey,\nThis malt whiskey shows some maple syrup and...,Jeffery Lindenmuth,750,45.2
4,Spirit Hound Colorado Honey Malt (Barrel No. 4...,85,65.0,American Malt Whiskey,\nStraight malt whiskey finished in barrels th...,Susannah Skiver Barton,750,45.0
...,...,...,...,...,...,...,...,...
5900,World Whiskey Society Goalong 5 year old Bourb...,86,150.0,World Whisky,\n\nThe first Chinese single malt to be export...,Susannah Skiver Barton,750,40.0
5901,"Sierra Norte Single Barrel White, 45%",86,50.0,World Whisky,\nWhite has the most familiar corn whiskey cha...,Jonny McCormick,750,45.0
5902,"Sierra Norte Single Barrel Purple Corn, 45%",85,50.0,World Whisky,\nPepper and allspice light up with savory bri...,Jonny McCormick,750,45.0
5903,That Boutique-y Whisky Company (distilled at S...,85,90.0,World Whisky,\nWhite pepper-dappled summer fruits on the no...,Jonny McCormick,750,52.5


In [292]:
Whis_df.to_csv('desktop/independant_projects/whiskey_data_scraper/clean_whiskey_data2.csv', index = False, encoding = 'utf-8-sig')